In [52]:
# Import the dependencies.

from flask import Flask, jsonify
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import datetime as dt

app = Flask(__name__)

#################################################
# Database Setup
#################################################

# reflect an existing database into a new model

engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)

# reflect the tables
@app.route("/")
def home():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end>"
    )

# Save references to each table
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Calculate the date 1 year ago from the last data point in the database
    one_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)

    # Query for the date and precipitation for the last year
    precipitation_data = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date >= one_year_ago).all()

    # Dict with date as the key and prcp as the value
    precip = {date: prcp for date, prcp in precipitation_data}
    return jsonify(precip)



#################################################
# Flask Setup
#################################################

@app.route("/api/v1.0/stations")
def stations():
    # Query for the stations
    results = session.query(Station.station).all()

    # Convert list of tuples into normal list
    stations = list(np.ravel(results))
    return jsonify(stations)

@app.route("/api/v1.0/tobs")
def tobs():
    # Calculate the date 1 year ago from the last data point in the database
    one_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)

    # Query the dates and temperature observations of the most active station for the last year of data
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == 'USC00519281').\
        filter(Measurement.date >= one_year_ago).all()

    # Convert list of tuples into normal list
    temps = list(np.ravel(results))
    return jsonify(temps)




#################################################
# Flask Routes
#################################################


@app.route("/api/v1.0/<start>")
def start_date(start):
    sl = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    results = session.query(*sel).filter(Measurement.date >= start).all()
    temps = list(np.ravel(results))
    return jsonify(temps)
    if temps['TMIN']:
        return jsonify(temps)
    else:
        return jsonify({"error": f"Date {start} not found or not formatted as YYYY-MM-DD."}), 404

@app.route("/api/v1.0/<start>/<end>")
def start_end_date(start, end):
    sl = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    results = session.query(*sel).filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    temps = list(np.ravel(results))
    return jsonify(temps)
    if temps['TMIN']: 
        return jsonify(temps)
    else:
        return jsonify({"error": f"Date {start} or {end} not found or not formatted as YYYY-MM-DD."}), 404


# Create our session (link) from Python to the DB

if __name__ == '__main__':
    app.run(debug=True)

%tb

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\macks\AppData\Local\Temp\ipykernel_29548\2549172655.py:19: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [15/Feb/2024 18:27:02] "GET / HTTP/1.1" 200 -


SystemExit: 1